be really clear about what's going into repaired tables: 
- repaired
- already OK
- unrepairable

Only ~ 215k repaired `env_package`s ?!
- yeah, there just aren't that many annotated in the biosample metadata


- some of this code should probably be moved back into the package
    - splitting and repiping 

- read in previous SSSOMs and don't re-map? or at least double check?

- don't forget to consult "model" field in biosample table too
    - for scoping, in addition to en_package and taxonomy_id?

- what to do about hits against obsolete classes... does OLS provide a mechanism to avoid?
- be careful with GAZ matches
- avoid IDless, OMIT, NCIT, MICRO, BTO

- is it worth trimming and concatenating "x1", "x2", "x3", where x is a mappable term? numbers may be low.
    - example = XXX

- reconcile "x biome" and "x" mappings
- take note of non-local salvage hits
    - like XXX
    - small/large (freshwater) lake?
    - include native labels in the review frame, in addition to local?
    - should use rdftab, not additional OLS calls
    

- switch from hard-coded `env_package_overrides` to something like SSSOM?
- mappings can come from merging strategies, first pass OLS or salvage OLS
- reflect those methods in SSSOM
- add string distance/confidence
    - multiple columns for multiple methods?

review these SSSOM columns

```
    "subject_category": [biosample_col_to_map] * results_rows,
    "predicate_id": ["skos:relatedMatch"] * results_rows,
    "match_type": ["Lexical"] * results_rows,
    "creator_id": ["https://github.com/turbomam/scoped-mapping"] * results_rows,
    #     "mapping_tool": ["https://www.ebi.ac.uk/ols/docs/api"] * results_rows,
    #     "confidence": 1 - frame_for_sssom["string_dist"],
```

- are there samples with a host_taxid and a (sample) taxonomy_id from "unclassified"?
- start at uncultured or metagenome taxonomy_id?

- what scoping on or mapping of  "gold" or "path"? WON'T FIND THAT IN NCBI BIOSAMPLE


What to save where? 
- new columns in biosample sqlite? 
- delimited file dumps
- move content from original column and insert the new values into that original column?
- pandas clipboard integration handy for review step


In [1]:
import pandas as pd
import re
import requests as requests
import sqlite3
import string
import urllib
import yaml

import scoped_mapping

from datetime import datetime
from pkg_resources import get_distribution, DistributionNotFound
from strsimpy.cosine import Cosine
from xml.etree import ElementTree
from tdda import rexpy

# Settings required from user:

See repo README for notes on setting up SQLite databases of OBO ontologies with semantic-sql, relation-graph and rdftab

In [2]:
# from https://www.ncbi.nlm.nih.gov/biosample/docs/packages/?format=xml
# see also https://www.ncbi.nlm.nih.gov/biosample/docs/packages/
biosample_packages_file = "../target/biosample_packages.xml"

# from ftp://ftp.ncbi.nlm.nih.gov//biosample/biosample_set.xml.gz
# via harmonized_table.db.gz
# in https://drive.google.com/drive/u/0/folders/1eL0v0stoduahjDpoDJIk3z2pJBAU4b2Y
biosample_sqlite_file = "../target/harmonized_table.db"
ncbitaxon_sqlite_file = "../semantic-sql/db/ncbitaxon.db"


# where do we require a single ontology and where can we use multiple?
primary_onto_prefix = "ENVO"
# 'gaz',
# need to rank results
additional_preferred_ontologies = ["po", "ncbitaxon"]

chars_to_whiteout = "._-"
my_query_fields = ""  # OLS weighted default
my_row_req = 3
my_string_dist_arg = 2
my_max_string_dist = 0.1
salvage_max_dist = 0.2

### What Biosample field should be mapped to ontology classes?

In [3]:
biosample_col_to_map = "env_local_scale"

### How should the mapping effort be scoped?
Additional taxonomic filters may be applied below

In [4]:
scoping_col = "env_package_normalization.EnvPackage"
scoping_value = "plant-associated"

### Settings for manual review of OLS search-based results
Results based on merging Biosample annotations to ontology classes by label or embedded ID are not exported for review at this time

In [5]:
# ols_review_file = "../local/ols_review.tsv"
ols_review_seperator = "\t"

strategy_col = "strategy"
include_col = "include"

first_pass_include_val = True
# first_pass_strategy_val = "env_braod_scale vs envo and gaz @ 0.1"
salvage_include_val = False
# salvage_strategy_val    = "env_braod_scale vs all of ols @ 0.2"

### Settings for SSSOM output

In [6]:
sssom_subject_prefix = biosample_col_to_map + "_" + scoping_col + "_" + scoping_value
sssom_file = sssom_subject_prefix + "_SSSOM.tsv"

In [7]:
# per_biosample_mapping_results_file = "per_biosample_scoped_mapping_results.tsv"

----

In [8]:
biosample_cnx = sqlite3.connect(biosample_sqlite_file)
ncbitaxon_cnx = sqlite3.connect(ncbitaxon_sqlite_file)
primary_ontology_sqlite_file = (
    "../semantic-sql/db/" + primary_onto_prefix.lower() + ".db"
)
primary_ontology_cnx = sqlite3.connect(primary_ontology_sqlite_file)
additional_preferred_ontologies.insert(0, primary_onto_prefix.lower())
additional_preferred_ontologies_str = ",".join(additional_preferred_ontologies)

In [9]:
q = "select * from id_patterns"
[id_patterns_frame, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)
id_patterns_frame

,ontology,id_pattern
0,BFO,BFO:\d{7}
1,CARO,CARO:\d{7}
2,CHEBI,"CHEBI:\d{4,6}"
3,ENVO,"ENVO:\d{7,8}"
4,FAO,FAO:0000001
5,FOODON,FOODON:\d{8}
6,GO,GO:\d{7}
7,IAO,IAO:\d{7}
8,NCBITaxon,NCBITaxon:\d+
9,OBI,OBI:\d{7}


In [10]:
id_patterns = dict(zip(id_patterns_frame.ontology, id_patterns_frame.id_pattern))
id_patterns

{'BFO': 'BFO:\\d{7}',
 'CARO': 'CARO:\\d{7}',
 'CHEBI': 'CHEBI:\\d{4,6}',
 'ENVO': 'ENVO:\\d{7,8}',
 'FAO': 'FAO:0000001',
 'FOODON': 'FOODON:\\d{8}',
 'GO': 'GO:\\d{7}',
 'IAO': 'IAO:\\d{7}',
 'NCBITaxon': 'NCBITaxon:\\d+',
 'OBI': 'OBI:\\d{7}',
 'PATO': 'PATO:\\d{7}',
 'PCO': 'PCO:\\d{7}',
 'PO': 'PO:\\d{7}',
 'RO': 'RO:0002577',
 'UBERON': 'UBERON:\\d{7}'}

- env_package = env_package annotation from NCBI Biosample file XXX
- count = number of biosamples using that env_package annotation
- lhs = checklist info
- rhs = potential package info
- extract = potential OBO ID from rhs column (currently harcoded and only looking for ENVO IDs)
- remaining_string = rhs/string, with potential OBO IDs removed
- remaining_tidied = remaining_string with case, whitespace and punctuation normailzastion
- rt_override = some remaining_tidied values can be replaced according to env_package_overrides
- EnvPackage = corresponding de-normalized value from package_dictionary
- is_canonical = false when EnvPackage is NaN

In [11]:
q = "select * from env_package_normalization"
[env_package_normalization, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx
)

env_package_normalization

,env_package,count,lhs,rhs,extract,remaining_string,remaining_tidied,rt_override,EnvPackage,is_canonical
0,None,14083847,,None,,,,,,1
1,host-associated,49254,,host-associated,,host-associated,host associated,host associated,host-associated,1
2,human-gut,47921,,human-gut,,human-gut,human gut,human gut,human-gut,1
3,water,16367,,water,,water,water,water,water,1
4,human-skin,13706,,human-skin,,human-skin,human skin,human skin,human-skin,1
...,...,...,...,...,...,...,...,...,...,...
87,env_package,1,,env_package,,env_package,env package,env package,None,0
88,gut microbiome,1,,gut microbiome,,gut microbiome,gut microbiome,gut microbiome,None,0
89,marine sediment (ENVO:00002113),1,,marine sediment (ENVO:00002113),,marine sediment (ENVO:00002113),marine sediment envo 00002113,marine sediment envo 00002113,None,0
90,saliva,1,,saliva,,saliva,saliva,saliva,None,0


## Get a table of MIxS annotations to be mapped to ontology classes.

Explicitly scope based on normalized package data. These values were set at the top of this notebook.


In [12]:
print(biosample_col_to_map)
print(scoping_col)
print(scoping_value)

env_local_scale
env_package_normalization.EnvPackage
plant-associated


**In this case, the scoping includes an inner join requirement for 'unclassified entities'**

In [13]:
q = (
    "select "
    + biosample_col_to_map
    + """, count(*) as count
from
    biosample b
join env_package_normalization on
    b.env_package = env_package_normalization.env_package
inner join biosample_sample_taxon_summary stic on
    b.taxonomy_id = stic.biosample_taxid
where """
    + scoping_col
    + " = '"
    + scoping_value
    + "' group by "
    + biosample_col_to_map
    + """
order by
    count(*) desc"""
)

[mapping_candidates, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

mapping_candidates

,env_local_scale,count
0,Root,558
1,rice field,310
2,Agar,270
3,lettuce field,220
4,Two-weeks old,219
...,...,...
121,12,1
122,11,1
123,10,1
124,1,1


----

## The Biosample format allows for pipe-delimited environmental package lists. 

Separate those out into their components.

----


In [14]:
multi_frames = []
for row in mapping_candidates.itertuples(index=True, name="PandaZ"):
    split_check = row[1]
    if split_check is None:
        split_check = ""
    splitted = pd.Series(split_check.split("|"))
    splitted_count = len(splitted)
    repeated = [split_check] * splitted_count
    repeated = pd.Series(repeated)
    as_frame = pd.DataFrame(dict(repeated=repeated, splitted=splitted)).reset_index()
    seq_list = list(range(1, splitted_count + 1))
    as_frame["part_count"] = splitted_count
    as_frame["seq"] = seq_list
    multi_frames.append(as_frame)
concat_frame = pd.concat(multi_frames)
concat_frame = concat_frame[["repeated", "splitted", "part_count", "seq"]]
concat_frame

,repeated,splitted,part_count,seq
0,Root,Root,1,1
0,rice field,rice field,1,1
0,Agar,Agar,1,1
0,lettuce field,lettuce field,1,1
0,Two-weeks old,Two-weeks old,1,1
...,...,...,...,...
0,12,12,1,1
0,11,11,1,1
0,10,10,1,1
0,1,1,1,1


In [15]:
mapping_candidates = mapping_candidates.merge(
    concat_frame, left_on=biosample_col_to_map, right_on="repeated", how="left"
)

mapping_candidates

,env_local_scale,count,repeated,splitted,part_count,seq
0,Root,558,Root,Root,1.0,1.0
1,rice field,310,rice field,rice field,1.0,1.0
2,Agar,270,Agar,Agar,1.0,1.0
3,lettuce field,220,lettuce field,lettuce field,1.0,1.0
4,Two-weeks old,219,Two-weeks old,Two-weeks old,1.0,1.0
...,...,...,...,...,...,...
121,12,1,12,12,1.0,1.0
122,11,1,11,11,1.0,1.0
123,10,1,10,10,1.0,1.0
124,1,1,1,1,1.0,1.0


## Normalize a few different ways `OBO` IDs have been entered
In the Biosample metadata

In [16]:
# this helps by standardizing what could be an ID
# but if the RHS is text, then envo should just be removed
mapping_candidates["term_tidy"] = mapping_candidates.splitted.str.replace(
    primary_onto_prefix.lower() + "[:_ ]",
    primary_onto_prefix + ":",
    regex=True,
    case=False,
)

mapping_candidates

,env_local_scale,count,repeated,splitted,part_count,seq,term_tidy
0,Root,558,Root,Root,1.0,1.0,Root
1,rice field,310,rice field,rice field,1.0,1.0,rice field
2,Agar,270,Agar,Agar,1.0,1.0,Agar
3,lettuce field,220,lettuce field,lettuce field,1.0,1.0,lettuce field
4,Two-weeks old,219,Two-weeks old,Two-weeks old,1.0,1.0,Two-weeks old
...,...,...,...,...,...,...,...
121,12,1,12,12,1.0,1.0,12
122,11,1,11,11,1.0,1.0,11
123,10,1,10,10,1.0,1.0,10
124,1,1,1,1,1.0,1.0,1


# Now try to extract ontology terms that are already present

In [17]:
candidate_series_decomposition = scoped_mapping.decompose_series(
    mapping_candidates["term_tidy"], id_patterns[primary_onto_prefix]
)

# candidate_series_decomposition

mapping_candidates = pd.concat(
    [mapping_candidates, candidate_series_decomposition], axis=1
)

mapping_candidates[
    "remaining_tidied"
] = mapping_candidates.remaining_tidied.str.replace(
    primary_onto_prefix.lower() + "[:_ ]", "", regex=True, case=False
)

mapping_candidates

,env_local_scale,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied
0,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root
1,rice field,310,rice field,rice field,1.0,1.0,rice field,rice field,,rice field,rice field
2,Agar,270,Agar,Agar,1.0,1.0,Agar,Agar,,Agar,agar
3,lettuce field,220,lettuce field,lettuce field,1.0,1.0,lettuce field,lettuce field,,lettuce field,lettuce field
4,Two-weeks old,219,Two-weeks old,Two-weeks old,1.0,1.0,Two-weeks old,Two-weeks old,,Two-weeks old,two weeks old
...,...,...,...,...,...,...,...,...,...,...,...
121,12,1,12,12,1.0,1.0,12,12,,12,12
122,11,1,11,11,1.0,1.0,11,11,,11,11
123,10,1,10,10,1.0,1.0,10,10,,10,10
124,1,1,1,1,1.0,1.0,1,1,,1,1


## Join the extracted IDs with their labels

_Start by connecting to the rdftab database from which the terms and label-like annotations will be obtained?_

## extracting the labels

In [18]:
q = "select stanza as subject, value from ids_labs_selected_ontolgies"

[onto_labels, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

onto_labels

,subject,value
0,BFO:0000001,entity
1,BFO:0000002,continuant
2,BFO:0000003,occurrent
3,BFO:0000004,independent continuant
4,BFO:0000006,spatial region
...,...,...
8350,PO:0030132,cincinnus inflorescence
8351,PO:0030133,compound cincinnus inflorescence
8352,PO:0030134,bostryx inflorescence
8353,PO:0030135,drepanium inflorescence


## and merging 

In [19]:
# mapping_candidates.columns

In [20]:
mapping_candidates = mapping_candidates.merge(
    onto_labels, left_on="extract", right_on="subject", how="left"
)
mapping_candidates

,env_local_scale,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,subject,value
0,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root,NaN,NaN
1,rice field,310,rice field,rice field,1.0,1.0,rice field,rice field,,rice field,rice field,NaN,NaN
2,Agar,270,Agar,Agar,1.0,1.0,Agar,Agar,,Agar,agar,NaN,NaN
3,lettuce field,220,lettuce field,lettuce field,1.0,1.0,lettuce field,lettuce field,,lettuce field,lettuce field,NaN,NaN
4,Two-weeks old,219,Two-weeks old,Two-weeks old,1.0,1.0,Two-weeks old,Two-weeks old,,Two-weeks old,two weeks old,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,12,1,12,12,1.0,1.0,12,12,,12,12,NaN,NaN
122,11,1,11,11,1.0,1.0,11,11,,11,11,NaN,NaN
123,10,1,10,10,1.0,1.0,10,10,,10,10,NaN,NaN
124,1,1,1,1,1.0,1.0,1,1,,1,1,NaN,NaN


## Use cosine string distance to see if the labels match closely enough

I.e. the labels claimed by the Biosample data set and the labels asserted in the ontology. if they're close enough, consider the assigned ID legit


_How close is close enough?_

In [21]:
my_cosine_obj = Cosine(my_string_dist_arg)
mapping_candidates["value"] = mapping_candidates["value"].fillna("")
mapping_candidates["remaining_tidied"] = mapping_candidates["remaining_tidied"].fillna(
    ""
)
mapping_candidates["cosine"] = mapping_candidates.apply(
    lambda my_row: my_cosine_obj.distance(
        my_row["remaining_tidied"].lower(), my_row["value"].lower()
    ),
    axis=1,
)
mapping_candidates

,env_local_scale,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,subject,value,cosine
0,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root,NaN,,1.0
1,rice field,310,rice field,rice field,1.0,1.0,rice field,rice field,,rice field,rice field,NaN,,1.0
2,Agar,270,Agar,Agar,1.0,1.0,Agar,Agar,,Agar,agar,NaN,,1.0
3,lettuce field,220,lettuce field,lettuce field,1.0,1.0,lettuce field,lettuce field,,lettuce field,lettuce field,NaN,,1.0
4,Two-weeks old,219,Two-weeks old,Two-weeks old,1.0,1.0,Two-weeks old,Two-weeks old,,Two-weeks old,two weeks old,NaN,,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,12,1,12,12,1.0,1.0,12,12,,12,12,NaN,,1.0
122,11,1,11,11,1.0,1.0,11,11,,11,11,NaN,,1.0
123,10,1,10,10,1.0,1.0,10,10,,10,10,NaN,,1.0
124,1,1,1,1,1.0,1.0,1,1,,1,1,NaN,,1.0


**Previously, we did a reality check on the claimed IDs and labels. If a label is claimed without any ID, that could still be a path to an ontology term.**

We'll be doing some merging, so make sure column names are meaningful and unique


In [22]:
mapping_candidates.columns = [
    biosample_col_to_map,
    "count",
    "repeated",
    "splitted",
    "part_count",
    "seq",
    "term_tidy",
    "string",
    "extract",
    "remaining_string",
    "remaining_tidied",
    "term_id",
    "lab_from_id",
    "lfi_cosine",
]
mapping_candidates = mapping_candidates.merge(
    onto_labels, left_on="remaining_tidied", right_on="value", how="left"
)

mapping_candidates.columns = [
    biosample_col_to_map,
    "count",
    "repeated",
    "splitted",
    "part_count",
    "seq",
    "term_tidy",
    "string",
    "extract",
    "remaining_string",
    "remaining_tidied",
    "term_id",
    "lab_from_id",
    "lfi_cosine",
    "term_id_from_lab",
    "value",
]


mapping_candidates

,env_local_scale,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,term_id,lab_from_id,lfi_cosine,term_id_from_lab,value
0,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root,NaN,,1.0,NCBITaxon:1,root
1,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root,NaN,,1.0,PO:0009005,root
2,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root,NaN,,1.0,PO:0009005,root
3,rice field,310,rice field,rice field,1.0,1.0,rice field,rice field,,rice field,rice field,NaN,,1.0,ENVO:00000296,rice field
4,Agar,270,Agar,Agar,1.0,1.0,Agar,Agar,,Agar,agar,NaN,,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,12,1,12,12,1.0,1.0,12,12,,12,12,NaN,,1.0,NaN,NaN
124,11,1,11,11,1.0,1.0,11,11,,11,11,NaN,,1.0,NaN,NaN
125,10,1,10,10,1.0,1.0,10,10,,10,10,NaN,,1.0,NaN,NaN
126,1,1,1,1,1.0,1.0,1,1,,1,1,NaN,,1.0,NaN,NaN


In [23]:
mapping_candidates = mapping_candidates.drop_duplicates()

mc_tifl_split = mapping_candidates["term_id_from_lab"].str.split(":")

import math

prefix_when_possible = []
for one_split in mc_tifl_split:
    if type(one_split) == list:
        os_first = one_split[0]
        prefix_when_possible.append(os_first)
    else:
        prefix_when_possible.append("")

prefix_when_possible = [each_string.lower() for each_string in prefix_when_possible]

prefix_when_possible

['ncbitaxon',
 'po',
 'envo',
 '',
 '',
 '',
 '',
 '',
 'envo',
 '',
 '',
 '',
 'envo',
 'envo',
 '',
 'envo',
 'envo',
 '',
 '',
 '',
 'envo',
 '',
 '',
 '',
 '',
 '',
 '',
 'envo',
 '',
 '',
 'envo',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'envo',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'envo',
 '',
 '',
 '',
 '',
 '',
 'envo',
 '',
 '',
 '',
 'envo',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [24]:
if "onto_prefix" in mapping_candidates.columns:
    mapping_candidates = mapping_candidates.drop("onto_prefix", 1)

mapping_candidates.loc[:, "onto_prefix"] = pd.Series(prefix_when_possible).values

mapping_candidates

/Users/MAM/Documents/gitrepos/scoped-mapping/sm_venv/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/MAM/Documents/gitrepos/scoped-mapping/sm_venv/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,env_local_scale,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,term_id,lab_from_id,lfi_cosine,term_id_from_lab,value,onto_prefix
0,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root,NaN,,1.0,NCBITaxon:1,root,ncbitaxon
1,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root,NaN,,1.0,PO:0009005,root,po
3,rice field,310,rice field,rice field,1.0,1.0,rice field,rice field,,rice field,rice field,NaN,,1.0,ENVO:00000296,rice field,envo
4,Agar,270,Agar,Agar,1.0,1.0,Agar,Agar,,Agar,agar,NaN,,1.0,NaN,NaN,
5,lettuce field,220,lettuce field,lettuce field,1.0,1.0,lettuce field,lettuce field,,lettuce field,lettuce field,NaN,,1.0,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,12,1,12,12,1.0,1.0,12,12,,12,12,NaN,,1.0,NaN,NaN,
124,11,1,11,11,1.0,1.0,11,11,,11,11,NaN,,1.0,NaN,NaN,
125,10,1,10,10,1.0,1.0,10,10,,10,10,NaN,,1.0,NaN,NaN,
126,1,1,1,1,1.0,1.0,1,1,,1,1,NaN,,1.0,NaN,NaN,


In [25]:
ranks = list(range(len(additional_preferred_ontologies)))

d = {"onto_prefix": additional_preferred_ontologies, "rank": ranks}
onto_rank_frame = pd.DataFrame(d)

onto_rank_frame

,onto_prefix,rank
0,envo,0
1,po,1
2,ncbitaxon,2


In [26]:
mapping_candidates = mapping_candidates.merge(onto_rank_frame, on="onto_prefix", how='left')

mapping_candidates

,env_local_scale,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,term_id,lab_from_id,lfi_cosine,term_id_from_lab,value,onto_prefix,rank
0,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root,NaN,,1.0,NCBITaxon:1,root,ncbitaxon,2.0
1,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root,NaN,,1.0,PO:0009005,root,po,1.0
2,rice field,310,rice field,rice field,1.0,1.0,rice field,rice field,,rice field,rice field,NaN,,1.0,ENVO:00000296,rice field,envo,0.0
3,Agar,270,Agar,Agar,1.0,1.0,Agar,Agar,,Agar,agar,NaN,,1.0,NaN,NaN,,NaN
4,lettuce field,220,lettuce field,lettuce field,1.0,1.0,lettuce field,lettuce field,,lettuce field,lettuce field,NaN,,1.0,NaN,NaN,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,12,1,12,12,1.0,1.0,12,12,,12,12,NaN,,1.0,NaN,NaN,,NaN
123,11,1,11,11,1.0,1.0,11,11,,11,11,NaN,,1.0,NaN,NaN,,NaN
124,10,1,10,10,1.0,1.0,10,10,,10,10,NaN,,1.0,NaN,NaN,,NaN
125,1,1,1,1,1.0,1.0,1,1,,1,1,NaN,,1.0,NaN,NaN,,NaN


In [27]:
best_maps = mapping_candidates.groupby(biosample_col_to_map)['rank'].min().reset_index()
best_maps

,env_local_scale,rank
0,1,NaN
1,10,NaN
2,11,NaN
3,12,NaN
4,13,NaN
...,...,...
120,rice field,0.0
121,soils used to inoculate seedlings,NaN
122,stream,0.0
123,sugar beet tissue,NaN


In [28]:
mapping_candidates = mapping_candidates.merge(best_maps, on=[biosample_col_to_map, "rank"])
mapping_candidates

,env_local_scale,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,remaining_tidied,term_id,lab_from_id,lfi_cosine,term_id_from_lab,value,onto_prefix,rank
0,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,root,NaN,,1.0,PO:0009005,root,po,1.0
1,rice field,310,rice field,rice field,1.0,1.0,rice field,rice field,,rice field,rice field,NaN,,1.0,ENVO:00000296,rice field,envo,0.0
2,Agar,270,Agar,Agar,1.0,1.0,Agar,Agar,,Agar,agar,NaN,,1.0,NaN,NaN,,NaN
3,lettuce field,220,lettuce field,lettuce field,1.0,1.0,lettuce field,lettuce field,,lettuce field,lettuce field,NaN,,1.0,NaN,NaN,,NaN
4,Two-weeks old,219,Two-weeks old,Two-weeks old,1.0,1.0,Two-weeks old,Two-weeks old,,Two-weeks old,two weeks old,NaN,,1.0,NaN,NaN,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,13,1,13,13,1.0,1.0,13,13,,13,13,NaN,,1.0,NaN,NaN,,NaN
121,12,1,12,12,1.0,1.0,12,12,,12,12,NaN,,1.0,NaN,NaN,,NaN
122,11,1,11,11,1.0,1.0,11,11,,11,11,NaN,,1.0,NaN,NaN,,NaN
123,10,1,10,10,1.0,1.0,10,10,,10,10,NaN,,1.0,NaN,NaN,,NaN


## Find consensus term IDs and labels


In [29]:
mapping_candidates["consensus_id"] = mapping_candidates["term_id_from_lab"]
mapping_candidates["consensus_lab"] = mapping_candidates["value"]

flag = mapping_candidates["consensus_id"].isnull() & (
    (
        ~mapping_candidates["term_id"].isnull()
        & mapping_candidates.lfi_cosine.le(my_max_string_dist)
    )
    | (
        ~mapping_candidates["term_id"].isnull()
        & mapping_candidates["remaining_tidied"].eq("")
    )
)

replacements = mapping_candidates.loc[flag, "term_id"]
mapping_candidates.loc[flag, "consensus_id"] = replacements

replacements = mapping_candidates.loc[flag, "lab_from_id"]
mapping_candidates.loc[flag, "consensus_lab"] = replacements

flag = mapping_candidates.consensus_id.isna()
antiflag = ~flag
mapping_candidates["id_or_lab_ok"] = antiflag

mapping_candidates["assembled_consensus"] = (
    mapping_candidates["consensus_lab"]
    + " ["
    + mapping_candidates["consensus_id"]
    + "]"
)

In [30]:
mapping_candidates

,env_local_scale,count,repeated,splitted,part_count,seq,term_tidy,string,extract,remaining_string,...,lab_from_id,lfi_cosine,term_id_from_lab,value,onto_prefix,rank,consensus_id,consensus_lab,id_or_lab_ok,assembled_consensus
0,Root,558,Root,Root,1.0,1.0,Root,Root,,Root,...,,1.0,PO:0009005,root,po,1.0,PO:0009005,root,True,root [PO:0009005]
1,rice field,310,rice field,rice field,1.0,1.0,rice field,rice field,,rice field,...,,1.0,ENVO:00000296,rice field,envo,0.0,ENVO:00000296,rice field,True,rice field [ENVO:00000296]
2,Agar,270,Agar,Agar,1.0,1.0,Agar,Agar,,Agar,...,,1.0,NaN,NaN,,NaN,NaN,NaN,False,NaN
3,lettuce field,220,lettuce field,lettuce field,1.0,1.0,lettuce field,lettuce field,,lettuce field,...,,1.0,NaN,NaN,,NaN,NaN,NaN,False,NaN
4,Two-weeks old,219,Two-weeks old,Two-weeks old,1.0,1.0,Two-weeks old,Two-weeks old,,Two-weeks old,...,,1.0,NaN,NaN,,NaN,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,13,1,13,13,1.0,1.0,13,13,,13,...,,1.0,NaN,NaN,,NaN,NaN,NaN,False,NaN
121,12,1,12,12,1.0,1.0,12,12,,12,...,,1.0,NaN,NaN,,NaN,NaN,NaN,False,NaN
122,11,1,11,11,1.0,1.0,11,11,,11,...,,1.0,NaN,NaN,,NaN,NaN,NaN,False,NaN
123,10,1,10,10,1.0,1.0,10,10,,10,...,,1.0,NaN,NaN,,NaN,NaN,NaN,False,NaN


## For which Biosample annotations were no easy mappings found?
How many Biosamples uses those annotations?

In [31]:
flag = ~mapping_candidates.id_or_lab_ok
needs_search = mapping_candidates.loc[flag]

needs_search = needs_search[["remaining_tidied", "count"]]

sum_by_needed = needs_search.groupby("remaining_tidied")["count"].sum()

sum_by_needed = sum_by_needed.to_frame()
sum_by_needed["remaining_tidied"] = sum_by_needed.index

sum_by_needed = sum_by_needed.sort_values("count", ascending=False)
sum_by_needed.reset_index(drop=True)

# sum_by_needed.to_sql("sum_by_needed", biosample_cnx, if_exists="replace", index=False)

,count,remaining_tidied
0,270,agar
1,220,lettuce field
2,219,two weeks old
3,207,plant associated habitat
4,198,beech forest
...,...,...
106,1,38
107,1,37
108,1,36
109,1,35


In [32]:
sum_by_needed

,count,remaining_tidied
remaining_tidied,,
agar,270,agar
lettuce field,220,lettuce field
two weeks old,219,two weeks old
plant associated habitat,207,plant associated habitat
beech forest,198,beech forest
...,...,...
38,1,38
37,1,37
36,1,36


## Extract the tidied strings

In [33]:
ebs_raw_list = list(sum_by_needed["remaining_tidied"])
ebs_raw_list.sort()

## Submit those tidied strings to a search engine

Specifically OLS search. This takes roughly one second per unique post-tidied submission

_Turn logging back on to show status?_
_Print the count and pre- and post- datestamps_


In [34]:
ebs_search_res = scoped_mapping.search_get_annotations_wrapper(
    ebs_raw_list,
    bad_chars=chars_to_whiteout,
    cat_name=biosample_col_to_map,
    ontoprefix=additional_preferred_ontologies_str,
    query_fields="",
    rr=5,
    string_dist_arg=my_string_dist_arg,
)
# ebs_search_res.to_sql(
#     "ols_search_results", biosample_cnx, if_exists="replace", index=False
# )

## Filter out the best of the acceptable mappings
From a string distance perspective

In [35]:
my_best_acceptable = scoped_mapping.get_best_acceptable(
    ebs_search_res, max_string_dist=my_max_string_dist
)
my_best_acceptable

,category,raw,query,name,string_dist_rank,string_dist,obo_id,label,search_rank,ontology_prefix,scope,type,iri,ontology_name
428,env_local_scale,coniferous forest,coniferous forest,coniferous forest,1,0.0,ENVO:01000433,needleleaf forest,1,ENVO,synonym,synonym,http://purl.obolibrary.org/obo/ENVO_01000433,envo
1282,env_local_scale,pool,pool,pool,1,0.0,ENVO:01000966,private swimming pool,2,ENVO,has_broad_synonym,annotation,http://purl.obolibrary.org/obo/ENVO_01000966,envo


----

## Filter out the submissions with no acceptable matches

In [36]:
no_acceptable_mappings = scoped_mapping.get_no_acceptable_mappings(
    ebs_search_res, my_best_acceptable
)

no_acceptable_mappings

,category,raw,query,name,string_dist_rank,string_dist,obo_id,label,search_rank,ontology_prefix,scope,type,iri,ontology_name
0,env_local_scale,1,1,Halomonas sp. 1-1,1,1.000,NCBITaxon:314588,Halomonas sp. 1-1,1,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_314588,ncbitaxon
1,env_local_scale,1,1,bacterium ST36-1-1,2,1.000,NCBITaxon:1705821,bacterium ST36-1-1,2,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_1705821,ncbitaxon
2,env_local_scale,1,1,Vibrio sp. 1-1,3,1.000,NCBITaxon:640334,Vibrio sp. 1-1,3,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_640334,ncbitaxon
3,env_local_scale,1,1,Streptomyces sp. 1-1,4,1.000,NCBITaxon:1119062,Streptomyces sp. 1-1,4,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_1119062,ncbitaxon
4,env_local_scale,1,1,Ralstonia sp. 1-1,5,1.000,NCBITaxon:269146,Ralstonia sp. 1-1,5,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_269146,ncbitaxon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589,env_local_scale,yacon fermentation,yacon fermentation,fermentation metagenome,1,0.311,NCBITaxon:1326787,fermentation metagenome,3,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_1326787,ncbitaxon
1590,env_local_scale,yacon fermentation,yacon fermentation,food fermentation metagenome,2,0.321,NCBITaxon:1154581,food fermentation metagenome,4,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_1154581,ncbitaxon
1587,env_local_scale,yacon fermentation,yacon fermentation,Yacon virus A,3,0.603,NCBITaxon:1868472,Yacon virus A,1,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_1868472,ncbitaxon
1588,env_local_scale,yacon fermentation,yacon fermentation,Yacon necrotic mottle virus,4,0.697,NCBITaxon:1561150,Yacon necrotic mottle virus,2,NCBITAXON,label,label,http://purl.obolibrary.org/obo/NCBITaxon_1561150,ncbitaxon


## Try searching the failures against all ontologies in OLS
what would it be like to use a local OLS instance? (latency-wise)

In [37]:
still_unmapped = list(set(list(no_acceptable_mappings["raw"])))
still_unmapped.sort()

print(len(still_unmapped))

still_unmapped

109


['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '5',
 '6',
 '7',
 '8',
 '9',
 'agar',
 'agricultural field or laboratory growth chamber',
 'agrigulutural field',
 'animal associated habitat',
 'beech forest',
 'blank',
 'cedar glade',
 'characterization of the microbial community present at vineyard',
 'chlorotic flecks on leaves',
 'crop',
 'crop fields',
 'disturbed field',
 'dna from roots of tree 14 13',
 'dna from roots of tree 14 14',
 'dna from roots of tree 14 15',
 'dna from roots of tree 16 10',
 'dna from roots of tree 16 11',
 'dna from roots of tree 16 12',
 'dna from roots of tree 18 7',
 'dna from roots of tree 20 2',
 'dna from roots of tree 20 3',
 'dna from roots of tree 20 6',
 'dna from roots of tree 24 8',
 'experimental pecan orchard',
 'experime

In [38]:
salvage_search_res = scoped_mapping.search_get_annotations_wrapper(
    still_unmapped,
    bad_chars="._-",
    cat_name="salvage",
    ontoprefix="",
    query_fields="",
    rr=5,
    string_dist_arg=2,
)

# salvage_search_res.to_sql(
#     "salvage_search_res", biosample_cnx, if_exists="replace", index=False
# )

## We appear to be at a point of diminishing returns
At the very least, will require some review

In [39]:
salvage_best_acceptable = scoped_mapping.get_best_acceptable(
    salvage_search_res, max_string_dist=salvage_max_dist
)
salvage_best_acceptable

,category,raw,query,name,string_dist_rank,string_dist,obo_id,label,search_rank,ontology_prefix,scope,type,iri,ontology_name
13,salvage,1,1,1,1,0.000,NCIT:C179011,COVID-19 Disease Severity 1,3,NCIT,hasExactSynonym,None,http://purl.obolibrary.org/obo/NCIT_C179011,ncit
29,salvage,10,10,10,1,0.000,NCIT:C105718,Karnofsky Performance Status 10,1,NCIT,synonym,synonym,http://purl.obolibrary.org/obo/NCIT_C105718,ncit
51,salvage,11,11,11,1,0.000,NCIT:C113425,Eleven,1,NCIT,synonym,synonym,http://purl.obolibrary.org/obo/NCIT_C113425,ncit
78,salvage,15,15,15,1,0.000,NCIT:C113429,Fifteen,5,NCIT,synonym,synonym,http://purl.obolibrary.org/obo/NCIT_C113429,ncit
98,salvage,16,16,16,1,0.000,NCIT:C14338,Human Papillomavirus Type 16,5,NCIT,hasExactSynonym,None,http://purl.obolibrary.org/obo/NCIT_C14338,ncit
107,salvage,17,17,17,1,0.000,NCIT:C113431,Seventeen,5,NCIT,synonym,synonym,http://purl.obolibrary.org/obo/NCIT_C113431,ncit
112,salvage,18,18,18,1,0.000,NCIT:C14377,Human Papillomavirus Type 18,1,NCIT,synonym,synonym,http://purl.obolibrary.org/obo/NCIT_C14377,ncit
132,salvage,19,19,19,1,0.000,NCIT:C105801,Nineteen,4,NCIT,synonym,synonym,http://purl.obolibrary.org/obo/NCIT_C105801,ncit
158,salvage,2,2,2,1,0.000,NCIT:C172969,Worry Score 2,5,NCIT,synonym,synonym,http://purl.obolibrary.org/obo/NCIT_C172969,ncit
163,salvage,20,20,20,1,0.000,NCIT:C105716,Karnofsky Performance Status 20,1,NCIT,synonym,synonym,http://purl.obolibrary.org/obo/NCIT_C105716,ncit


## Save the mappings for review!

_Note that the merge on ID or label mappings are not included here_


In [60]:
my_best_acceptable[include_col] = first_pass_include_val
salvage_best_acceptable[include_col] = salvage_include_val
best_and_salvage = pd.concat([my_best_acceptable, salvage_best_acceptable])

best_and_salvage["ols_curation_notes"] = ""

# best_and_salvage.to_csv(ols_review_file, sep=ols_review_seperator, index=False)

best_and_salvage = best_and_salvage[
    [
        "category",
        "raw",
        "query",
        "name",
        "obo_id",
        "label",
        "ontology_prefix",
        "include",
        "ols_curation_notes",
        "scope",
        "type",
        "iri",
        "ontology_name",
        "string_dist_rank",
        "string_dist",
        "search_rank",
    ]
]

best_and_salvage = best_and_salvage.sort_values(['include','ontology_prefix'])

In [62]:
best_and_salvage.to_clipboard(index=False)

In [ ]:
raise SystemExit("Don't skip reviewing and saving!")

## Do some review in a spreadsheet application!
Specifically, columns `include` and `ols_curation_notes`

Then Save

----

## Now read the reviewed spreadsheet back in

In [ ]:
# curated = pd.read_csv(ols_review_file, sep=ols_review_seperator)
# curated

## OR copy and paste

In [64]:
curated = pd.read_clipboard()
curated

,category,raw,query,name,obo_id,label,ontology_prefix,include,ols_curation_notes,scope,type,iri,ontology_name,string_dist_rank,string_dist,search_rank
0,salvage,blank,blank,blank,AFRL:0000259,blank role,AFO,False,NaN,synonym,synonym,http://purl.allotrope.org/ontologies/role#AFRL...,afo,1,0.000,1
1,salvage,inoculum,inoculum,inoculum,ARO:3004417,inoculum,ARO,False,NaN,label,label,http://purl.obolibrary.org/obo/ARO_3004417,aro,1,0.000,1
2,salvage,beech forest,beech forest,beech forest soil,ENVO:00005770,beech forest soil,ENVO,False,NaN,label,label,http://purl.obolibrary.org/obo/ENVO_00005770,envo,1,0.171,1
3,salvage,urediniospore tissue,urediniospore tissue,urediniospore,FAO:0000026,urediniospore,FAO,False,NaN,label,label,http://purl.obolibrary.org/obo/FAO_0000026,fao,1,0.184,1
4,salvage,forest,forest,Forest,GAZ:00454366,Forest,GAZ,False,NaN,label,label,http://purl.obolibrary.org/obo/GAZ_00454366,gaz,1,0.000,1
5,salvage,highlands,highlands,Highlands,GAZ:00246323,Highlands,GAZ,False,NaN,label,label,http://purl.obolibrary.org/obo/GAZ_00246323,gaz,1,0.000,1
6,salvage,oak tree,oak tree,Oak Tree,GAZ:00454476,Oak Tree,GAZ,False,NaN,label,label,http://purl.obolibrary.org/obo/GAZ_00454476,gaz,1,0.000,1
7,salvage,oulu,oulu,Oulu,GAZ:00010238,Oulu,GAZ,False,NaN,label,label,http://purl.obolibrary.org/obo/GAZ_00010238,gaz,1,0.000,1
8,salvage,organic feature,organic feature,organic feature,ENVO:01000159,organic feature,MICRO,False,NaN,label,label,http://purl.obolibrary.org/obo/ENVO_01000159,micro,1,0.000,1
9,salvage,1,1,1,NCIT:C179011,COVID-19 Disease Severity 1,NCIT,False,NaN,hasExactSynonym,NaN,http://purl.obolibrary.org/obo/NCIT_C179011,ncit,1,0.000,3


## Filter on `include`

In [65]:
flag = curated["include"]
curated = curated.loc[flag]

curated.columns = "ols_" + curated.columns
# curated.to_sql("curated", biosample_cnx, if_exists="replace", index=False)
curated

,ols_category,ols_raw,ols_query,ols_name,ols_obo_id,ols_label,ols_ontology_prefix,ols_include,ols_ols_curation_notes,ols_scope,ols_type,ols_iri,ols_ontology_name,ols_string_dist_rank,ols_string_dist,ols_search_rank
47,salvage,glasshouse,glasshouse,glasshouse,AGRO:00000363,greenhouse,AGRO,True,NaN,synonym,synonym,http://purl.obolibrary.org/obo/AGRO_00000363,agro,1,0.0,1
48,salvage,greenhouse,greenhouse,greenhouse,AGRO:00000363,greenhouse,AGRO,True,NaN,label,label,http://purl.obolibrary.org/obo/AGRO_00000363,agro,1,0.0,1
49,env_local_scale,coniferous forest,coniferous forest,coniferous forest,ENVO:01000433,needleleaf forest,ENVO,True,NaN,synonym,synonym,http://purl.obolibrary.org/obo/ENVO_01000433,envo,1,0.0,1
50,env_local_scale,pool,pool,pool,ENVO:01000966,private swimming pool,ENVO,True,NaN,has_broad_synonym,annotation,http://purl.obolibrary.org/obo/ENVO_01000966,envo,1,0.0,2


In [66]:
merge_search_merged = mapping_candidates.merge(
    curated, how="outer", left_on="remaining_tidied", right_on="ols_raw"
)

In [67]:
flag = (
    ~merge_search_merged["id_or_lab_ok"]
    & ~merge_search_merged["ols_obo_id"].eq("")
    & ~merge_search_merged["ols_obo_id"].isna()
)

replacement = merge_search_merged.loc[flag, "ols_obo_id"]

merge_search_merged.loc[flag, "consensus_id"] = replacement

replacement = merge_search_merged.loc[flag, "ols_label"]

merge_search_merged.loc[flag, "consensus_lab"] = replacement

replacement = (
    merge_search_merged.loc[flag, "consensus_lab"]
    + " ["
    + merge_search_merged.loc[flag, "consensus_id"]
    + "]"
)

merge_search_merged.loc[flag, "assembled_consensus"] = replacement

In [68]:
# merge_search_merged.to_sql(
#     "merge_search_merged", biosample_cnx, if_exists="replace", index=False
# )

In [69]:
summary = merge_search_merged[[biosample_col_to_map, "count", "assembled_consensus"]]
summary = summary.sort_values("count", ascending=False)
summary

,env_local_scale,count,assembled_consensus
0,Root,558,root [PO:0009005]
1,rice field,310,rice field [ENVO:00000296]
2,Agar,270,NaN
3,lettuce field,220,NaN
4,Two-weeks old,219,NaN
...,...,...,...
69,DNA from roots of tree 24_8,1,NaN
68,Highlands,1,NaN
67,Lowlands,1,NaN
66,Mountain,1,mountain [ENVO:00000081]


In [70]:
frame_for_sssom = merge_search_merged

na_flag = frame_for_sssom["splitted"].isna()
frame_for_sssom = frame_for_sssom[~na_flag]

na_flag = frame_for_sssom["consensus_id"].isna()
frame_for_sssom = frame_for_sssom[~na_flag]

raw_queries = list(frame_for_sssom["splitted"])
urlencodeds = list(map(urllib.parse.quote, raw_queries))
urlencodeds = [sssom_subject_prefix + ":" + item for item in urlencodeds]

results_rows = len(frame_for_sssom.index)
iso8601_stamp = datetime.now().replace(microsecond=0).isoformat()

sssom_frame = {
    "subject_category": [biosample_col_to_map] * results_rows,
    "subject_label": frame_for_sssom["splitted"],
    "predicate_id": ["skos:relatedMatch"] * results_rows,
    "object_id": frame_for_sssom["consensus_id"],
    "object_label": frame_for_sssom["consensus_lab"],
    "match_type": ["Lexical"] * results_rows,
    "creator_id": ["https://github.com/turbomam/scoped-mapping"] * results_rows,
    #     "mapping_tool": ["https://www.ebi.ac.uk/ols/docs/api"] * results_rows,
    "mapping_date": [iso8601_stamp] * results_rows,
    #     "confidence": 1 - frame_for_sssom["string_dist"],
    "subject_id": urlencodeds,
}

sssom_frame = pd.DataFrame(sssom_frame)

sssom_frame.to_csv(sssom_file, sep=ols_review_seperator, index=False)

sssom_frame

,subject_category,subject_label,predicate_id,object_id,object_label,match_type,creator_id,mapping_date,subject_id
0,env_local_scale,Root,skos:relatedMatch,PO:0009005,root,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T18:33:50,env_local_scale_env_package_normalization.EnvP...
1,env_local_scale,rice field,skos:relatedMatch,ENVO:00000296,rice field,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T18:33:50,env_local_scale_env_package_normalization.EnvP...
7,env_local_scale,garden,skos:relatedMatch,ENVO:00000011,garden,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T18:33:50,env_local_scale_env_package_normalization.EnvP...
8,env_local_scale,coniferous forest,skos:relatedMatch,ENVO:01000433,needleleaf forest,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T18:33:50,env_local_scale_env_package_normalization.EnvP...
11,env_local_scale,prairie,skos:relatedMatch,ENVO:00000260,prairie,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T18:33:50,env_local_scale_env_package_normalization.EnvP...
12,env_local_scale,field,skos:relatedMatch,ENVO:01000352,field,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T18:33:50,env_local_scale_env_package_normalization.EnvP...
14,env_local_scale,vineyard,skos:relatedMatch,ENVO:00000116,vineyard,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T18:33:50,env_local_scale_env_package_normalization.EnvP...
15,env_local_scale,Vineyard,skos:relatedMatch,ENVO:00000116,vineyard,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T18:33:50,env_local_scale_env_package_normalization.EnvP...
16,env_local_scale,stream,skos:relatedMatch,ENVO:00000023,stream,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T18:33:50,env_local_scale_env_package_normalization.EnvP...
17,env_local_scale,pool,skos:relatedMatch,ENVO:01000966,private swimming pool,Lexical,https://github.com/turbomam/scoped-mapping,2021-06-11T18:33:50,env_local_scale_env_package_normalization.EnvP...


# From here down requires refactoring and generalization 

That may even be true for some of the blocks above this!

## Join together pipe-delimited env_broad_scale mappings 
_Any other constraints? If part_count is > 1, do we need to check for empty or null env_broad_scale, repeated or splitted?_

In [71]:
multi_part_flag = merge_search_merged["part_count"] > 1
multi_part_flag.value_counts()

False    125
Name: part_count, dtype: int64

In [72]:
multi_part_frame = merge_search_merged.loc[multi_part_flag]

multi_part_frame = multi_part_frame[
    [
        biosample_col_to_map,
        "count",
        "repeated",
        "splitted",
        "part_count",
        "seq",
        "consensus_id",
        "consensus_lab",
    ]
]

multi_part_frame = multi_part_frame.sort_values(
    [biosample_col_to_map, "seq"], ascending=(True, True)
)

multi_part_frame

,env_local_scale,count,repeated,splitted,part_count,seq,consensus_id,consensus_lab


In [73]:
part_count = multi_part_frame.groupby([biosample_col_to_map], sort=False)[
    "part_count"
].max()
part_count.value_counts()

Series([], Name: part_count, dtype: int64)

In [74]:
seq_max = multi_part_frame.groupby([biosample_col_to_map], sort=False)["seq"].max()
seq_max

Series([], Name: seq, dtype: float64)

In [75]:
parts_check = pd.DataFrame(dict(part_count=part_count, seq_max=seq_max)).reset_index()

all_parts_flag = parts_check["seq_max"] == parts_check["part_count"]
ebs_with_all_parts = parts_check.loc[all_parts_flag, biosample_col_to_map]
ebs_with_all_parts

all_parts_flag = multi_part_frame[biosample_col_to_map].isin(list(ebs_with_all_parts))
all_parts_frame = multi_part_frame.loc[all_parts_flag]
all_parts_frame

,env_local_scale,count,repeated,splitted,part_count,seq,consensus_id,consensus_lab


In [76]:
unique_pipesep_ebs = list(set(list(all_parts_frame[biosample_col_to_map])))
unique_pipesep_ebs.sort()
repipe_dict_list = []
for one_pipesep in unique_pipesep_ebs:
    flag = all_parts_frame[biosample_col_to_map] == one_pipesep
    temp_frame = all_parts_frame.loc[flag]
    ids_list = list(temp_frame["consensus_id"])
    lab_list = list(temp_frame["consensus_lab"])
    s = "|"
    ids_repipe = s.join(ids_list)
    lab_repipe = s.join(lab_list)

    repipe_dict = {
        biosample_col_to_map: one_pipesep,
        "consensus_id": ids_repipe,
        "consensus_lab": lab_repipe,
    }
    repipe_dict_list.append(repipe_dict)
repipe_frame = pd.DataFrame(repipe_dict_list)
repipe_frame

""


In [77]:
sssom_min_cols = sssom_frame[["subject_label", "object_id", "object_label"]]

sssom_min_cols.columns = [biosample_col_to_map, "consensus_id", "consensus_lab"]

sssom_min_cols

,env_local_scale,consensus_id,consensus_lab
0,Root,PO:0009005,root
1,rice field,ENVO:00000296,rice field
7,garden,ENVO:00000011,garden
8,coniferous forest,ENVO:01000433,needleleaf forest
11,prairie,ENVO:00000260,prairie
12,field,ENVO:01000352,field
14,vineyard,ENVO:00000116,vineyard
15,Vineyard,ENVO:00000116,vineyard
16,stream,ENVO:00000023,stream
17,pool,ENVO:01000966,private swimming pool


In [78]:
repipe_rows = len(repipe_frame.index)
if repipe_rows > 0:
    direct_and_repipe = pd.concat([temp, repipe_frame])
else:
    direct_and_repipe = sssom_min_cols
direct_and_repipe

,env_local_scale,consensus_id,consensus_lab
0,Root,PO:0009005,root
1,rice field,ENVO:00000296,rice field
7,garden,ENVO:00000011,garden
8,coniferous forest,ENVO:01000433,needleleaf forest
11,prairie,ENVO:00000260,prairie
12,field,ENVO:01000352,field
14,vineyard,ENVO:00000116,vineyard
15,Vineyard,ENVO:00000116,vineyard
16,stream,ENVO:00000023,stream
17,pool,ENVO:01000966,private swimming pool


In [79]:
direct_and_repipe.to_sql(
    "direct_and_repipe", biosample_cnx, if_exists="replace", index=False
)

## refactor this even more?


In [80]:
q = (
    """
select
	b.id, b."""
    + biosample_col_to_map
    + """,
	dar.consensus_id ,
	dar.consensus_lab
from
	direct_and_repipe dar
left join biosample b on
	b."""
    + biosample_col_to_map
    + """ = dar."""
    + biosample_col_to_map
    + """
inner join biosample_sample_taxon_summary stic on
	b.taxonomy_id = stic.biosample_taxid
join env_package_normalization on
	b.env_package = env_package_normalization.env_package
where """
    + scoping_col
    + " = '"
    + scoping_value
    + "'"
)

[per_biosample_scoped_mapping_results, query_duration] = scoped_mapping.timed_query(
    q, biosample_cnx, print_timing=True
)

2021-06-11 18:33:51.065664
2021-06-11 18:33:51.210968
0:00:00.145304


In [81]:
per_biosample_scoped_mapping_results["notes"] = scoping_col + "=" + scoping_value

table_name = "repaired_" + biosample_col_to_map

per_biosample_scoped_mapping_results.to_sql(
    table_name, biosample_cnx, if_exists="append", index=False
)

per_biosample_scoped_mapping_results

,id,env_local_scale,consensus_id,consensus_lab,notes
0,BIOSAMPLE:SAMEA1695842,field,ENVO:01000352,field,env_package_normalization.EnvPackage=plant-ass...
1,BIOSAMPLE:SAMEA1695808,field,ENVO:01000352,field,env_package_normalization.EnvPackage=plant-ass...
2,BIOSAMPLE:SAMEA1695787,field,ENVO:01000352,field,env_package_normalization.EnvPackage=plant-ass...
3,BIOSAMPLE:SAMEA1695839,field,ENVO:01000352,field,env_package_normalization.EnvPackage=plant-ass...
4,BIOSAMPLE:SAMEA1695833,field,ENVO:01000352,field,env_package_normalization.EnvPackage=plant-ass...
...,...,...,...,...,...
1457,BIOSAMPLE:SAMN02628361,Vineyard,ENVO:00000116,vineyard,env_package_normalization.EnvPackage=plant-ass...
1458,BIOSAMPLE:SAMN02640034,rhizosphere,ENVO:00005801,rhizosphere,env_package_normalization.EnvPackage=plant-ass...
1459,BIOSAMPLE:SAMN02641154,greenhouse,AGRO:00000363,greenhouse,env_package_normalization.EnvPackage=plant-ass...
1460,BIOSAMPLE:SAMN02664684,Mountain,ENVO:00000081,mountain,env_package_normalization.EnvPackage=plant-ass...
